In [8]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSemanticSegmentation, TrainingArguments, Trainer

import torch

import pandas as pd
from konlpy.tag import Mecab
import re
from tqdm import tqdm, tqdm_notebook
from kiwipiepy import Kiwi

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report



# 1차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
service_data = survey[['content_id', 'service']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
service_data['content_id'] = service_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
service_data = service_data[service_data['content_id'] != 0].reset_index().drop(columns=['index'])
service_data['content_id'] = service_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(service_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
service_data = pd.merge(service_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
service_data['service'] = service_data['service'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

service_data['content'] = service_data['content'].apply(cleaned_content)

final_df = service_data[['content', 'service']]

# 라벨별 개수 확인
print(final_df['service'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['service'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['service'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_69361/4145494998.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  service_data['content_id'] = service_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


service
 0    4884
 1      89
-1      50
Name: count, dtype: int64


train_data service
 0    3906
 1      70
-1      42
Name: count, dtype: int64

 test_data service
 0    977
 1     14
-1     14
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005




중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [3]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [5]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["service"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['service'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './service_model/check_point/try_1',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [6]:
# train 진행
trainer.train() 

  0%|          | 0/2510 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_68495/726067504.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 20%|█▉        | 500/2510 [02:04<08:16,  4.05it/s]

{'loss': 0.1746, 'learning_rate': 4.00398406374502e-05, 'epoch': 1.99}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_68495/726067504.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 40%|███▉      | 1000/2510 [04:06<06:06,  4.12it/s]

{'loss': 0.0636, 'learning_rate': 3.00796812749004e-05, 'epoch': 3.98}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_68495/726067504.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 60%|█████▉    | 1500/2510 [06:07<03:59,  4.22it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_68495/726067504.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|█████████▉| 2500/2510 [10:09<00:02,  4.23it/s]

{'loss': 0.0269, 'learning_rate': 1.99203187250996e-07, 'epoch': 9.96}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_68495/726067504.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 2510/2510 [10:12<00:00,  4.10it/s]

{'train_runtime': 612.1111, 'train_samples_per_second': 65.397, 'train_steps_per_second': 4.101, 'train_loss': 0.06815159260276779, 'epoch': 10.0}


TrainOutput(global_step=2510, training_loss=0.06815159260276779, metrics={'train_runtime': 612.1111, 'train_samples_per_second': 65.397, 'train_steps_per_second': 4.101, 'train_loss': 0.06815159260276779, 'epoch': 10.0})

In [7]:
trainer.save_model("./service_model/try_1/service_model1")
tokenizer.save_pretrained("./service_model/try_1/service_tokenizer1")

('./service_model/try_1/service_tokenizer1/tokenizer_config.json',
 './service_model/try_1/service_tokenizer1/special_tokens_map.json',
 './service_model/try_1/service_tokenizer1/vocab.txt',
 './service_model/try_1/service_tokenizer1/added_tokens.json')

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./service_model/try_1/service_model1"
tokenizer_path = "./service_model/try_1/service_tokenizer1"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 1001/1001 [00:13<00:00, 72.04it/s]


,content,service,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,0,0
...,...,...,...
999,나에겐 너무 어려웠던. 방탈짬바 있는 분들에게 추천,0,0
1000,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0,0
1001,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0,0
1002,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,0,0


In [5]:
print(final_df['service'].value_counts())

service
 0    4884
 1      89
-1      50
Name: count, dtype: int64


In [6]:
print(train_data['service'].value_counts())
print(test_data['service'].value_counts())

service
 0    3891
 1      70
-1      42
Name: count, dtype: int64
service
 0    973
 1     14
-1     14
Name: count, dtype: int64


In [7]:
print(classification_report(test_data['service'], test_data['pred']))
test_data2 = test_data[test_data['service'] != 0]
print(classification_report(test_data2['service'], test_data2['pred']))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        14
           0       1.00      1.00      1.00       973
           1       0.52      0.93      0.67        14

    accuracy                           0.98      1001
   macro avg       0.51      0.64      0.55      1001
weighted avg       0.98      0.98      0.98      1001

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        14
           0       0.00      0.00      0.00         0
           1       0.54      0.93      0.68        14

    accuracy                           0.46        28
   macro avg       0.18      0.31      0.23        28
weighted avg       0.27      0.46      0.34        28



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [8]:
test = [
    '서비스 대박',
    '개친절함',
    '서비스 개좋음',
    '서비스 무슨 일',
    '서비스 불친절하네',
    '불친절함',
    '스토리 쓰레기네',
    '서비스는 좋은데 재미없었어요.',
]

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10)

문장: 서비스 대박, 평가: 1
----------
문장: 개친절함, 평가: 1
----------
문장: 서비스 개좋음, 평가: 1
----------
문장: 서비스 무슨 일, 평가: 1
----------
문장: 서비스 불친절하네, 평가: 1
----------
문장: 불친절함, 평가: 1
----------
문장: 스토리 쓰레기네, 평가: 1
----------
문장: 서비스는 좋은데 재미없었어요., 평가: 2
----------


# 2차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
service_data = survey[['content_id', 'service']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
service_data['content_id'] = service_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
service_data = service_data[service_data['content_id'] != 0].reset_index().drop(columns=['index'])
service_data['content_id'] = service_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(service_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
service_data = pd.merge(service_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
service_data['service'] = service_data['service'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

service_data['content'] = service_data['content'].apply(cleaned_content)

final_df = service_data[['content', 'service']]

# 라벨별 개수 확인
print(final_df['service'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['service'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['service'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_70688/4145494998.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  service_data['content_id'] = service_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


service
 0    4884
 1      89
-1      50
Name: count, dtype: int64


train_data service
 0    3906
 1      70
-1      42
Name: count, dtype: int64

 test_data service
 0    977
 1     14
-1     14
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005




중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [3]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [4]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["service"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['service'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './service_model/check_point/try_2',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 15,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [5]:
# train 진행
trainer.train() 

  0%|          | 0/3765 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_69506/1575091855.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 13%|█▎        | 500/3765 [02:04<13:13,  4.12it/s]

{'loss': 0.1715, 'learning_rate': 4.335989375830013e-05, 'epoch': 1.99}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_69506/1575091855.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 27%|██▋       | 1000/3765 [04:06<11:26,  4.03it/s]

{'loss': 0.0701, 'learning_rate': 3.671978751660027e-05, 'epoch': 3.98}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_69506/1575091855.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 40%|███▉      | 1500/3765 [06:07<09:02,  4.17it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_69506/1575091855.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 93%|█████████▎| 3500/3765 [14:16<01:04,  4.09it/s]

{'loss': 0.0254, 'learning_rate': 3.51925630810093e-06, 'epoch': 13.94}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_69506/1575091855.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 3765/3765 [15:21<00:00,  4.08it/s]

{'train_runtime': 921.7939, 'train_samples_per_second': 65.139, 'train_steps_per_second': 4.084, 'train_loss': 0.055353891137111706, 'epoch': 15.0}


TrainOutput(global_step=3765, training_loss=0.055353891137111706, metrics={'train_runtime': 921.7939, 'train_samples_per_second': 65.139, 'train_steps_per_second': 4.084, 'train_loss': 0.055353891137111706, 'epoch': 15.0})

In [6]:
trainer.save_model("./service_model/try_2/service_model2")
tokenizer.save_pretrained("./service_model/try_2/service_tokenizer2")

('./service_model/try_2/service_tokenizer2/tokenizer_config.json',
 './service_model/try_2/service_tokenizer2/special_tokens_map.json',
 './service_model/try_2/service_tokenizer2/vocab.txt',
 './service_model/try_2/service_tokenizer2/added_tokens.json')

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./service_model/try_2/service_model2"
tokenizer_path = "./service_model/try_2/service_tokenizer2"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 1001/1001 [00:16<00:00, 61.20it/s]


,content,service,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,0,0
...,...,...,...
999,나에겐 너무 어려웠던. 방탈짬바 있는 분들에게 추천,0,0
1000,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0,0
1001,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0,0
1002,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,0,0


In [5]:
print('전체 라벨 수', final_df['service'].value_counts())
print('train_data 라벨 수', train_data['service'].value_counts())
print('test_data 라벨 수', test_data['service'].value_counts())

전체 라벨 수 service
 0    4884
 1      89
-1      50
Name: count, dtype: int64
train_data 라벨 수 service
 0    3891
 1      70
-1      42
Name: count, dtype: int64
test_data 라벨 수 service
 0    973
 1     14
-1     14
Name: count, dtype: int64


In [6]:
print('0포함 예측률',classification_report(test_data['service'], test_data['pred']))
test_data2 = test_data[test_data['service'] != 0]
print('0제외 예측률',classification_report(test_data2['service'], test_data2['pred']))

0포함 예측률               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        14
           0       1.00      1.00      1.00       973
           1       0.52      0.93      0.67        14

    accuracy                           0.99      1001
   macro avg       0.51      0.64      0.56      1001
weighted avg       0.98      0.99      0.98      1001

0제외 예측률               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        14
           0       0.00      0.00      0.00         0
           1       0.52      0.93      0.67        14

    accuracy                           0.46        28
   macro avg       0.17      0.31      0.22        28
weighted avg       0.26      0.46      0.33        28



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [7]:
test = [
    '서비스 대박',
    '개친절함',
    '서비스 개좋음',
    '서비스 무슨 일',
    '서비스 불친절하네',
    '불친절함',
    '스토리 쓰레기네',
    '서비스는 좋은데 재미없었어요.',
]

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10)

문장: 서비스 대박, 평가: 1
----------
문장: 개친절함, 평가: 1
----------
문장: 서비스 개좋음, 평가: 2
----------
문장: 서비스 무슨 일, 평가: 2
----------
문장: 서비스 불친절하네, 평가: 2
----------
문장: 불친절함, 평가: 1
----------
문장: 스토리 쓰레기네, 평가: 1
----------
문장: 서비스는 좋은데 재미없었어요., 평가: 2
----------


## 2차 결과
- 데이터 넣어주기!
- 데이터가 부족해서 결과가 안 나온당

# 3차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
service_data = survey[['content_id', 'service']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
service_data['content_id'] = service_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
service_data = service_data[service_data['content_id'] != 0].reset_index().drop(columns=['index'])
service_data['content_id'] = service_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(service_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
service_data = pd.merge(service_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
service_data['service'] = service_data['service'].astype(int)

final_df = service_data[['content', 'service']]

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

final_df['content'] = final_df['content'].apply(cleaned_content)
final_df['content'] = final_df['content'].apply(kiwi_clean)


final_df = final_df.drop_duplicates()
final_df = final_df.reset_index().drop(columns = ['index'])

# 라벨별 개수 확인
print(final_df['service'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['service'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['service'].value_counts())

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_96083/2105697174.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  service_data['content_id'] = service_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)
/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_96083/2105697174.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['content'] = final_df['content'].apply(cleaned_content)


service
 0    4753
 1      89
-1      50
Name: count, dtype: int64


train_data service
 0    3807
 1      65
-1      42
Name: count, dtype: int64

 test_data service
 0    950
 1     14
-1     14
Name: count, dtype: int64


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_96083/2105697174.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['content'] = final_df['content'].apply(kiwi_clean)


In [6]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [7]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["service"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['service'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './service_model/check_point/try_3',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 30,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [8]:
# train 진행
trainer.train() 

  0%|          | 0/7350 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_93641/1452544400.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  7%|▋         | 500/7350 [01:47<28:02,  4.07it/s]  

{'loss': 0.1809, 'learning_rate': 4.6598639455782315e-05, 'epoch': 2.04}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_93641/1452544400.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 14%|█▎        | 1000/7350 [03:48<26:02,  4.06it/s]

{'loss': 0.0913, 'learning_rate': 4.319727891156463e-05, 'epoch': 4.08}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_93641/1452544400.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 20%|██        | 1500/7350 [05:47<23:15,  4.19it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_93641/1452544400.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 95%|█████████▌| 7000/7350 [27:56<01:23,  4.21it/s]

{'loss': 0.0543, 'learning_rate': 2.3809523809523808e-06, 'epoch': 28.57}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_93641/1452544400.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 7350/7350 [29:21<00:00,  4.17it/s]

{'train_runtime': 1761.155, 'train_samples_per_second': 66.672, 'train_steps_per_second': 4.173, 'train_loss': 0.08718719274819303, 'epoch': 30.0}


TrainOutput(global_step=7350, training_loss=0.08718719274819303, metrics={'train_runtime': 1761.155, 'train_samples_per_second': 66.672, 'train_steps_per_second': 4.173, 'train_loss': 0.08718719274819303, 'epoch': 30.0})

In [9]:
trainer.save_model("./service_model/try_3/service_model3")
tokenizer.save_pretrained("./service_model/try_3/service_tokenizer3")

('./service_model/try_3/service_tokenizer3/tokenizer_config.json',
 './service_model/try_3/service_tokenizer3/special_tokens_map.json',
 './service_model/try_3/service_tokenizer3/vocab.txt',
 './service_model/try_3/service_tokenizer3/added_tokens.json')

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./service_model/try_3/service_model3"
tokenizer_path = "./service_model/try_3/service_tokenizer3"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 978/978 [00:14<00:00, 66.52it/s]


,content,service,pred
0,인 활동 있다 추락 조심,0,0
1,인 나 눈 없다 치다,0,0
2,인테리어 향기 굿 문제 전반 쉽다 문제 이다 같다 나 능지 얼마나 낮다 알다 있다,0,0
3,문제 재밌다 타임 어택 느낌 좋다 급하다 하다 스토리 못 즐기다 아쉽다,0,0
4,첫 탈출 우왕좌왕 그래도 거의 탈출 코앞 이다,0,0
...,...,...,...
973,나 너무 어렵다 방탈짬바 있다 추천,0,0
974,인 각자 하다 뿌듯하다 테마,0,0
975,첫 디버프로 초반 절다 보다 못 보다 힌트 쓰다 잘 꾸미다 살짝 장치 오류 아쉽다,0,0
976,인 볼륨 압도 히터 방 빵빵하다 다른 테마 궁금,0,0


In [8]:
print(classification_report(test_data['service'], test_data['pred']))
test_data2 = test_data[test_data['service'] != 0]
print(classification_report(test_data2['service'], test_data2['pred']))

/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        14
           0       0.98      1.00      0.99       950
           1       0.55      0.43      0.48        14

    accuracy                           0.98       978
   macro avg       0.51      0.48      0.49       978
weighted avg       0.96      0.98      0.97       978

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        14
           0       0.00      0.00      0.00         0
           1       0.60      0.43      0.50        14

    accuracy                           0.21        28
   macro avg       0.20      0.14      0.17        28
weighted avg       0.30      0.21      0.25        28



In [9]:
test = [
    '서비스 대박',
    '개친절함',
    '서비스 개좋음',
    '서비스 무슨 일',
    '서비스 불친절하네',
    '불친절함',
    '스토리 쓰레기네',
    '서비스는 좋은데 재미없었어요.',
]

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10)

문장: 서비스 대박, 평가: 1
----------
문장: 개친절함, 평가: 2
----------
문장: 서비스 개좋음, 평가: 1
----------
문장: 서비스 무슨 일, 평가: 1
----------
문장: 서비스 불친절하네, 평가: 2
----------
문장: 불친절함, 평가: 2
----------
문장: 스토리 쓰레기네, 평가: 1
----------
문장: 서비스는 좋은데 재미없었어요., 평가: 1
----------


# 4차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
service_data = survey[['content_id', 'service']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
service_data['content_id'] = service_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
service_data = service_data[service_data['content_id'] != 0].reset_index().drop(columns=['index'])
service_data['content_id'] = service_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(service_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
service_data = pd.merge(service_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
service_data['service'] = service_data['service'].astype(int)
final_df = service_data[['content', 'service']]

# 데이터 추가
new_data = [
    ('서비스 적당함', 1), ('서비스 아주 딱이었음', 1), ('서비스가 친절해서 좋았어요', 1), ('그냥 왕 친절함', 1),
    ('서비스 완벽했음', 1), ('온도 습도 밝기 서비스 다 완벽했다', 1), ('처음부터 끝까지 다 친절해서 좋았어요', 1), ('친절 굳', 1),
    ('서비스 적당함, 방탈출 과정에서 필요한 서비스가 딱 적절함', 1), ('서비스 아주 딱이었음, 방탈출에서의 서비스가 완벽함', 1),
    ('서비스가 친절해서 좋았어요, 방탈출 도중 높은 서비스 수준에 만족', 1), ('그냥 왕 친절함, 방탈출에서 받은 서비스가 정말 왕 친절', 1),
    ('서비스 완벽했음, 방탈출에서의 서비스가 탁월함', 1), ('온도 습도 밝기 서비스 다 완벽했다, 방탈출 공간에서 제공된 서비스가 완벽함', 1),
    ('처음부터 끝까지 다 친절해서 좋았어요, 방탈출 도중 받은 서비스가 시작부터 끝까지 친절함', 1), ('친절 굳, 방탈출에서의 서비스 친절도가 굉장히 좋음', 1),
    ('서비스가 훌륭해서 더 재미있었어요, 방탈출에서의 서비스가 전반적인 경험을 더욱 풍부하게 해줌', 1), ('서비스에 감동 받았어요, 방탈출에서의 서비스가 감동적이었음', 1),
    ('서비스가 예상보다 좋아서 기분 좋았어요, 방탈출에서의 서비스가 예상을 뛰어넘어 기분 좋음', 1), ('서비스가 매우 세심하게 이뤄져서 좋았어요, 방탈출에서의 세심한 서비스가 만족스러움', 1),
    ('서비스가 기대 이상이었어요, 방탈출에서의 서비스가 기대를 뛰어넘어 만족스러움', 1), ('서비스가 팀의 플레이를 더 재미있게 만들어줬어요, 방탈출에서의 서비스가 팀 플레이를 더욱 즐겁게 함', 1),
    ('서비스가 고급스러워서 놀랐어요, 방탈출에서의 고급스러운 서비스가 놀라움', 1), ('서비스가 전문적이어서 신뢰감이 있었어요, 방탈출에서의 전문적인 서비스가 신뢰감을 부여함', 1),
    ('서비스가 훌륭하게 조화를 이루고 있어서 좋았어요, 방탈출에서의 서비스가 조화롭게 어우러져 좋음', 1), ('서비스가 편안한 분위기를 만들어줘서 좋았어요, 방탈출에서의 서비스가 분위기를 더욱 편안하게 함', 1),
    ('서비스가 최고라서 다음에도 꼭 방문하고 싶어졌어요, 방탈출에서의 최고의 서비스로 인해 재방문을 강하게 희망', 1), ('서비스가 기억에 남을 만큼 좋았어요, 방탈출에서의 서비스가 기억에 남음', 1),

    ('서비스 개쓰레기임', -1), ('서비스 실화냐', -1), ('너무 불친절해서 다시는 안 가고 싶음', -1), ('돈 받고 이러는 거 아니지', -1),
    ('서비스 너무 별로여서 하루종일 불쾌했어요.', -1), ('서비스 개실망', -1), ('직원 교육 안 시키냐', -1), ('서비스 최악임', -1), 
    ('엄청 불친절하고 별로임', -1), ('안 친절함', -1), ('친절하지 않음', -1), ('친절이 없음', -1), ('직원들 싸가지 없음', -1),
    ('이렇게 해서 어떻게 장사하려고 함? 친절이라곤 하나도 모르는 사람들이잖아.', -1),
    ('서비스 나쁘다', -1), ('서비스 개쓰레기임, 방탈출에서의 서비스가 실망스러움', -1), ('서비스 실화냐, 방탈출에서의 서비스가 현실과 너무 다름', -1),
    ('너무 불친절해서 다시는 안 가고 싶음, 방탈출 도중 경험한 불친절한 서비스로 인해 재방문 의사 없음', -1),
    ('돈 받고 이러는 거 아니지, 지불한 대가에 비해 서비스가 부족함', -1),
    ('서비스 너무 별로여서 하루종일 불쾌했어요., 방탈출에서의 서비스가 매우 불쾌함을 느껴 하루 종일 기분 나쁨', -1),
    ('서비스 개실망, 방탈출에서의 서비스에 대한 실망이 큼', -1),
    ('직원 교육 안 시키냐, 방탈출의 서비스 수준이 직원 교육이 부족한 것 같음', -1),
    ('서비스 최악임, 방탈출에서 경험한 서비스가 최악임', -1),
    ('서비스 나쁘다, 방탈출에서의 서비스 품질이 낮음', -1),
    ('이런 서비스면 다른 데로 가야지, 방탈출에서의 서비스가 다른 곳으로 가야할 정도로 나쁨', -1),
    ('서비스가 망했어, 방탈출에서의 서비스가 전반적으로 망침', -1),
    ('서비스가 형편없어, 방탈출에서의 서비스 퀄리티가 형편없음', -1),
    ('이 정도면 돈이 아깝다, 방탈출에서의 서비스가 지불한 돈에 비해 가치 없음', -1),
    ('서비스가 최악인데 어떻게 운영하냐, 방탈출에서의 최악의 서비스로 운영에 대한 의문', -1),
    ('이런 서비스에 돈 주기 싫다, 방탈출에서의 서비스가 돈을 주기에는 미흡함', -1),
    ('서비스가 말도 안 되게 나쁘다, 방탈출에서의 서비스 품질이 심각하게 나쁨', -1),
    ('서비스가 엉망이야, 방탈출에서의 서비스가 전반적으로 엉망임', -1),
    ('이런 서비스로 어떻게 운영이 되냐, 방탈출에서의 서비스가 운영에 대한 의문을 남김', -1),
    ('서비스가 최악이었어, 방탈출에서의 서비스가 최악임', -1),
    ('서비스가 너무 별로라 실망스러웠어요, 방탈출에서의 서비스에 대한 실망이 큼', -1),
    ('서비스가 너무 나쁘다, 방탈출에서의 서비스 품질이 매우 낮음', -1)
]

new_df = pd.DataFrame(new_data, columns=['content', 'service'])
final_df = pd.concat([final_df, new_df])





# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

final_df['content'] = final_df['content'].apply(cleaned_content)
final_df['content'] = final_df['content'].apply(kiwi_clean)


final_df = final_df.drop_duplicates()
final_df = final_df.reset_index().drop(columns = ['index'])

# 라벨별 개수 확인
print(final_df['service'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['service'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['service'].value_counts())

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_2297/823715976.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  service_data['content_id'] = service_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


service
 0    4753
 1     117
-1      86
Name: count, dtype: int64


train_data service
 0    3809
 1      87
-1      69
Name: count, dtype: int64

 test_data service
 0    901
-1     49
 1     41
Name: count, dtype: int64


In [12]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [8]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["service"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['service'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './service_model/check_point/try_4',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 40,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [9]:
# train 진행
trainer.train() 

  0%|          | 0/9920 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_99827/972789307.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  5%|▌         | 500/9920 [01:28<28:11,  5.57it/s]  

{'loss': 0.0111, 'learning_rate': 4.7479838709677423e-05, 'epoch': 2.02}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_99827/972789307.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 10%|█         | 1000/9920 [02:55<26:04,  5.70it/s]

{'loss': 0.0125, 'learning_rate': 4.495967741935484e-05, 'epoch': 4.03}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_99827/972789307.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 15%|█▌        | 1500/9920 [04:23<25:34,  5.49it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_99827/972789307.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 96%|█████████▌| 9500/9920 [27:26<01:12,  5.80it/s]

{'loss': 0.0004, 'learning_rate': 2.1169354838709676e-06, 'epoch': 38.31}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_99827/972789307.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 9920/9920 [28:40<00:00,  5.77it/s]

{'train_runtime': 1720.2693, 'train_samples_per_second': 92.195, 'train_steps_per_second': 5.767, 'train_loss': 0.003199645265516254, 'epoch': 40.0}


TrainOutput(global_step=9920, training_loss=0.003199645265516254, metrics={'train_runtime': 1720.2693, 'train_samples_per_second': 92.195, 'train_steps_per_second': 5.767, 'train_loss': 0.003199645265516254, 'epoch': 40.0})

In [11]:
trainer.save_model("./service_model/try_4/service_model4")
tokenizer.save_pretrained("./service_model/try_4/service_tokenizer4")

('./service_model/try_4/service_tokenizer4/tokenizer_config.json',
 './service_model/try_4/service_tokenizer4/special_tokens_map.json',
 './service_model/try_4/service_tokenizer4/vocab.txt',
 './service_model/try_4/service_tokenizer4/added_tokens.json')

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./service_model/try_4/service_model4"
tokenizer_path = "./service_model/try_4/service_tokenizer4"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

  0%|          | 0/991 [00:00<?, ?it/s]

100%|██████████| 991/991 [00:11<00:00, 84.59it/s]


,content,service,pred
0,쉽재테마 스토리 테마 진행 자체 구성 단순 좋다 연출 보다 재미 잘 끌어올리다,0,0
1,인 귀엽다 근데 방 탈출 아니다 무슨 체험 카페 오다 같다,0,0
2,인 생각 어렵다 문제 난이도 따지다 그 달동네 이미지 세탁소 비하다 맵다 맛 그렇지...,0,0
3,덜덜거리다 장치 잠깐잠깐 창공,0,0
4,쉽다 재밌다 스토리 취향 인 추천 드리다,0,0
...,...,...,...
986,서비스 엉망 이다 방 탈출 서비스 전반 엉망 이다,-1,-1
987,이런 서비스 어떻다 운영 되다 방 탈출 서비스 운영 대하다 의문 남기다,-1,-1
988,서비스 최악 이다 방 탈출 서비스 최악 이다,-1,-1
989,서비스 너무 별로 이다 실망 방 탈출 서비스 대하다 실망 크다,-1,-1


In [5]:
print(classification_report(test_data['service'], test_data['pred']))
test_data2 = test_data[test_data['service'] != 0]
print(classification_report(test_data2['service'], test_data2['pred']))

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00        49
           0       1.00      1.00      1.00       901
           1       0.98      0.98      0.98        41

    accuracy                           1.00       991
   macro avg       0.99      0.99      0.99       991
weighted avg       1.00      1.00      1.00       991

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00        49
           0       0.00      0.00      0.00         0
           1       1.00      0.98      0.99        41

    accuracy                           0.99        90
   macro avg       0.67      0.66      0.66        90
weighted avg       1.00      0.99      0.99        90



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [6]:
test = [
    '서비스 대박',
    '개친절함',
    '서비스 개좋음',
    '서비스 무슨 일',
    '서비스 불친절하네',
    '불친절함',
    '스토리 쓰레기네',
    '서비스는 좋은데 재미없었어요.',
]

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10)

문장: 서비스 대박, 평가: 2
----------
문장: 개친절함, 평가: 0
----------
문장: 서비스 개좋음, 평가: 2
----------
문장: 서비스 무슨 일, 평가: 2
----------
문장: 서비스 불친절하네, 평가: 0
----------
문장: 불친절함, 평가: 0
----------
문장: 스토리 쓰레기네, 평가: 1
----------
문장: 서비스는 좋은데 재미없었어요., 평가: 2
----------
